In [1]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#globals
equities = ['SPY', 'IWM', 'QQQ', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR']
fixed_income = ['TLT', 'IEF']
precious_metals = ['GLD', 'GDX', 'SLV', 'GDXJ']

tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

In [2]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

def get_benchmark(indices,start, end):
    indices = indices

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

def ticker_data(tickers, start, end):
    days = (end-start).days

    index = pd.date_range(start, periods = days, freq='D')
    df = pd.DataFrame(index = index,columns = tickers)

    for ticker in tickers:
        df[ticker] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    df = df.dropna()
    return(df)

def portfolio_data(portfolio, start, end, rf, leverage, margin_int):
    
    weights = np.array(list(portfolio.values()))
    tickers = list(portfolio.keys())
    data = ticker_data(tickers, start, end)
    mean_returns = data.pct_change().mean()
    cov = data.pct_change().cov()
    
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std

    portfolio_stats = {'ret': portfolio_return, 'stdev':portfolio_std, 'sharpe': sharpe_ratio,
                      'levered ret': levered_return, 'levered stdev':levered_std, 'levered sharpe': levered_sharpe}
    
    portfolio_df = pd.DataFrame(portfolio_stats,columns=['ret','stdev','sharpe'] +list(portfolio.keys())+ ['levered ret','levered stdev','levered sharpe'], index = [0])
    
    for ticker in tickers:
        portfolio_df[ticker] = portfolio[ticker]
    
    return(portfolio_df)

def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df

#simulate leveraged portfolios

def calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std
    
    return portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe

def simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int):
    results_matrix = np.zeros((len(mean_returns)+6, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe = calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = levered_return
        results_matrix[4,i] = levered_std
        results_matrix[5,i] = levered_sharpe
        
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+6,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe','levered ret','levered stdev','levered sharpe'] + [ticker for ticker in tickers])
        
    return results_df

#find target leverage

def apply_leverage(input_portfolio, leverage, margin_int):
    
    results_frame = input_portfolio.copy()
    results_frame['levered ret']=[0.0]*len(results_frame)
    results_frame['levered stdev']=[0.0]*len(results_frame)
    results_frame['levered sharpe'] = [0.0]*len(results_frame)
    
    for i in range(len(results_frame)):
        results_frame['levered ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
        results_frame['levered stdev'][i] = results_frame['stdev'][i] * leverage
        results_frame['levered sharpe'][i] = results_frame['levered ret'][i]/results_frame['levered stdev'][i]
        
    return(results_frame)

def target_risk(portfolios, acceptable_risk, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    leverage = float(acceptable_risk / optimal_portfolio['stdev'])
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def target_return(portfolios, tgt_rtn, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    
    leverage = (tgt_rtn - margin_int)/(float(optimal_portfolio['ret']-margin_int))
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)

def series_corr(ser1, ser2):
    correlation = ser1.pct_change().corr(ser2.pct_change())
    return(correlation)

def create_corr_matrix(ticker_df):
    data = ticker_df
    stock_dict = ticker_df.to_dict(orient='series')
    corr_matrix = data.corr()

    for stock1 in stock_dict.keys():
        for stock2 in stock_dict.keys():
            corr_matrix[stock1][stock2] = series_corr(stock_dict[stock1], stock_dict[stock2])
    return(corr_matrix)


In [ ]:
indices = ['SPY', 'IWM', 'VT', 'VEF.TO', 'VTI']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
standard = get_benchmark(indices,start, end)
standard

In [11]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
ticker_df = ticker_data(tickers, start, end)
ticker_df.head()

,SPY,GLD,TLT,LQD
2005-07-25,90.983475,42.509998,56.903557,59.293636
2005-07-26,91.094292,42.250000,56.995152,59.304337
2005-07-27,91.426628,42.439999,56.903557,59.229378
2005-07-28,92.002747,42.750000,57.410233,59.454315
2005-07-29,91.389702,42.820000,56.842503,59.320442


In [10]:
portfolio = {'TLT':0.402064, 'VTI':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.015

portfolio_DF = portfolio_data(portfolio, start, end, rf, leverage, margin_int)
portfolio_DF

,ret,stdev,sharpe,TLT,VTI,GLD,LQD,levered ret,levered stdev,levered sharpe
0,0.090287,0.080004,1.128538,0.402064,0.293876,0.157595,0.146465,0.127931,0.120006,1.066041


In [11]:
tickers = ['TLT', 'VTI', 'GLD', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
results_frame.head()

,ret,stdev,sharpe,TLT,VTI,GLD,LQD
0,0.091018,0.082149,1.107964,0.440205,0.356134,0.097563,0.106097
1,0.093621,0.104873,0.892704,0.307703,0.082228,0.462420,0.147649
2,0.085246,0.077077,1.105988,0.260556,0.212274,0.195473,0.331697
3,0.096555,0.089533,1.078424,0.323606,0.346180,0.270072,0.060142
4,0.092937,0.099339,0.935554,0.060566,0.357510,0.305086,0.276838


In [14]:
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['sharpe']==max_sharpe]


,ret,stdev,sharpe,TLT,VTI,GLD,LQD
5810,0.09039,0.08008,1.128749,0.396332,0.295928,0.160329,0.147411


In [476]:
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104145,0.199594,0.521784
2,IWM,0.0968185,0.24876,0.389205
3,VT,0.0850438,0.22475,0.378392
4,QQQ,0.157312,0.214189,0.734452


In [3]:
tickers = ['SPY', 'IWM', 'QQQ', 'GDX','VTI', 'GLD', 'USO', 'TLT', 'LQD', 'IEF']
#start = date.today() - timedelta(days = 3650*1.5)
#end = date.today()

start = dt.datetime(2015,1,1)
end = dt.datetime(2020,6,30)


ticker_dframe = ticker_data(tickers, start, end)
corr_matrix = create_corr_matrix(ticker_dframe)

In [4]:
corr_matrix

,SPY,IWM,QQQ,GDX,VTI,GLD,USO,TLT,LQD,IEF
SPY,1.000000,0.904668,0.936037,0.080634,0.997218,-0.055132,0.379840,-0.415951,0.190936,-0.424131
IWM,0.904668,1.000000,0.817994,0.084534,0.926534,-0.058194,0.358263,-0.389279,0.189671,-0.404532
QQQ,0.936037,0.817994,1.000000,0.052136,0.928820,-0.053227,0.309337,-0.372750,0.144391,-0.388391
GDX,0.080634,0.084534,0.052136,1.000000,0.084188,0.755698,0.148258,0.257397,0.263263,0.291476
VTI,0.997218,0.926534,0.928820,0.084188,1.000000,-0.052615,0.381630,-0.412803,0.197075,-0.421418
GLD,-0.055132,-0.058194,-0.053227,0.755698,-0.052615,1.000000,0.007616,0.343822,0.334581,0.406094
USO,0.379840,0.358263,0.309337,0.148258,0.381630,0.007616,1.000000,-0.226420,0.040368,-0.227552
TLT,-0.415951,-0.389279,-0.372750,0.257397,-0.412803,0.343822,-0.226420,1.000000,0.509420,0.935094
LQD,0.190936,0.189671,0.144391,0.263263,0.197075,0.334581,0.040368,0.509420,1.000000,0.521275
IEF,-0.424131,-0.404532,-0.388391,0.291476,-0.421418,0.406094,-0.227552,0.935094,0.521275,1.000000


In [ ]:
def sim_port(holdings, ticker_df):